In [19]:
global t
t = 0

TERRAINS = {
    "W": 1,  
    "G": 1.2,
    "T": min(1.5, 0.1 * t),
    "M": 1.6,
}

In [20]:
def heuristic(a, b):
    """Manhattan distance multiplied by the lowest terrain cost (water)."""
    (x1, y1), (x2, y2) = a, b
    return (abs(x1 - x2) + abs(y1 - y2)) * TERRAINS["W"]

In [21]:
import os
import time

def print_grid(opened, closed, path):
    os.system('cls' if os.name == 'nt' else 'clear')  # Clear terminal
    print("\nGrid:")
    for r in range(ROWS):
        row_repr = ""
        for c in range(COLS):
            cell = grid[r][c]
            if (r, c) == start:
                row_repr += "S "
            elif (r, c) == goal:
                row_repr += "G "
            elif (r, c) in path:
                row_repr += "* "
            elif (r, c) in opened:
                row_repr += "O "
            elif (r, c) in closed:
                row_repr += "X "
            else:
                row_repr += cell + " "
        print(row_repr)
    
    print("\nOpened list:")
    print(opened)

    print("\nClosed list:")
    print(closed)

    print("\nCurrent path (partial):")
    print(path)

    time.sleep(0.05)  # Slow down for step-by-step visualization

# First Grid:

In [22]:
grid = [
    list("WGGTG"),
    list("GMWMM"),
    list("MTMWG"),
    list("GGGMM"),
    list("MTMGW"),
    list("GWGMG"),
    list("GGMTM"),
    list("WGGWM"),
]

ROWS, COLS = len(grid), len(grid[0])
start = (0, 0)  
goal = (6, 4)   

In [23]:
def neighbors(node):
    x, y = node
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < ROWS and 0 <= ny < COLS:
            if TERRAINS[grid[nx][ny]] is not None:
                yield (nx, ny)

In [24]:
import heapq

def a_star(start, goal):
    global t
    open_heap = []
    heapq.heappush(open_heap, (0, start))

    came_from = {}
    g_score = {start: 0}

    opened_order, closed_order = [], []

    while open_heap:
        current_f, current = heapq.heappop(open_heap)
        closed_order.append(current)
        t += 1

        if current == goal:                     
            break                               

        for n in neighbors(current):
            terrain_cost = TERRAINS[grid[n[0]][n[1]]]
            tentative_g = g_score[current] + terrain_cost

            if n not in g_score or tentative_g < g_score[n]:
                came_from[n] = current
                g_score[n] = tentative_g
                f = tentative_g + heuristic(n, goal)
                heapq.heappush(open_heap, (f, n))
                if n not in opened_order:
                    opened_order.append(n)

    path = []
    node = goal
    while node in came_from or node == start:
        path.append(node)
        if node == start:
            break
        node = came_from[node]
    path.reverse()

    total_cost = g_score.get(goal)
    return path, opened_order, closed_order, total_cost

In [25]:
def reconstruct_path(came_from, node):
    path = [node]
    while node in came_from:
        node = came_from[node]
        path.append(node)
    path.reverse()
    return path

In [26]:
import heapq

def a_star_visual(start, goal):
    global t
    open_heap = []
    heapq.heappush(open_heap, (0, start))

    came_from = {}
    g_score = {start: 0}

    opened_set = set()
    closed_set = set()
    t = 0

    while open_heap:
        current_f, current = heapq.heappop(open_heap)
        if current in closed_set:
            continue

        closed_set.add(current)
        t += 1

        # Print current visual state
        if current != start:
            path = reconstruct_path(came_from, current)
        else:
            path = [start]
        print_grid(opened_set, closed_set, path)

        if current == goal:
            print("\nGoal reached!")
            final_path = reconstruct_path(came_from, goal)
            total_cost = g_score.get(goal)
            return final_path, list(opened_set), list(closed_set), total_cost

        for neighbor in neighbors(current):
            if neighbor in closed_set:
                continue

            terrain_cost = TERRAINS[grid[neighbor[0]][neighbor[1]]]
            tentative_g = g_score[current] + terrain_cost

            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f = tentative_g + heuristic(neighbor, goal)
                heapq.heappush(open_heap, (f, neighbor))
                opened_set.add(neighbor)

    print("No path found.")
    return [], list(opened_set), list(closed_set), None

In [27]:
path, opened, closed, cost = a_star(start, goal)

In [28]:
cost

9.4

In [29]:
path_ = a_star_visual(start, goal)


Grid:
S G G T G 
G M W M M 
M T M W G 
G G G M M 
M T M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
set()

Closed list:
{(0, 0)}

Current path (partial):
[(0, 0)]

Grid:
S * G T G 
O M W M M 
M T M W G 
G G G M M 
M T M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 0), (0, 1)}

Closed list:
{(0, 1), (0, 0)}

Current path (partial):
[(0, 0), (0, 1)]

Grid:
S O O T G 
* O W M M 
M T M W G 
G G G M M 
M T M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 0), (0, 2), (1, 1), (0, 1)}

Closed list:
{(0, 1), (1, 0), (0, 0)}

Current path (partial):
[(0, 0), (1, 0)]

Grid:
S * * T G 
O O W M M 
O T M W G 
G G G M M 
M T M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(0, 1), (1, 1), (2, 0), (0, 2), (1, 0)}

Closed list:
{(0, 1), (1, 0), (0, 2), (0, 0)}

Current path (partial):
[(0, 0), (0, 1), (0, 2)]

Grid:
S * * * G 
O O O M M 
O T M W G 
G G G M M 
M T M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(0, 1), (1, 2), (1, 1), (0, 3), (2, 0), (0, 2), (

In [30]:
cost

9.4

In [31]:
print("\nGrid:")
for r in range(ROWS):
    row_repr = ""
    for c in range(COLS):
        symbol = grid[r][c]
        if (r, c) == start:
            symbol = "S"
        elif (r, c) == goal:
            symbol = "G"
        elif (r, c) in path:
            symbol = "*"
        row_repr += symbol + " "
    print(row_repr)

print("\nOpened nodes (order added):")
print(opened)

print("\nClosed nodes (order processed):")
print(closed)

print("\nFinal path from S to G:")
print(path)



Grid:
S * G T G 
G * W M M 
M * M W G 
G * G M M 
M * M G W 
G * * * G 
G G M * G 
W G G W M 

Opened nodes (order added):
[(1, 0), (0, 1), (1, 1), (0, 2), (2, 0), (1, 2), (0, 3), (1, 3), (0, 4), (1, 4), (2, 3), (3, 3), (2, 2), (2, 4), (3, 4), (4, 3), (3, 2), (2, 1), (3, 1), (4, 1), (3, 0), (5, 1), (4, 0), (4, 2), (6, 1), (5, 0), (5, 2), (6, 2), (5, 3), (7, 1), (6, 0), (4, 4), (5, 4), (6, 3), (7, 3), (6, 4)]

Closed nodes (order processed):
[(0, 0), (0, 1), (1, 0), (0, 2), (0, 3), (0, 4), (1, 3), (2, 3), (1, 4), (2, 4), (1, 2), (3, 3), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (5, 2), (6, 1), (4, 2), (4, 3), (4, 4), (5, 3), (6, 3), (6, 4)]

Final path from S to G:
[(0, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (5, 2), (5, 3), (6, 3), (6, 4)]


# Inadmissible Heuristic

If you select, for example, the highest terrain cost as the multiplier for the heuristic, A* will overestimate the cost, leading to a loss of optimality

In [32]:
def heuristic(a, b):
    """Manhattan distance multiplied by the lowest terrain cost (water)."""
    (x1, y1), (x2, y2) = a, b
    return (abs(x1 - x2) + abs(y1 - y2)) * TERRAINS["M"]

In [33]:
path, opened, closed, cost = a_star(start, goal)

In [34]:
print("\nGrid:")
for r in range(ROWS):
    row_repr = ""
    for c in range(COLS):
        symbol = grid[r][c]
        if (r, c) == start:
            symbol = "S"
        elif (r, c) == goal:
            symbol = "G"
        elif (r, c) in path:
            symbol = "*"
        row_repr += symbol + " "
    print(row_repr)

print("\nOpened nodes (order added):")
print(opened)

print("\nClosed nodes (order processed):")
print(closed)

print("\nFinal path from S to G:")
print(path)


Grid:
S * * * * 
G M W M * 
M T M W * 
G G G M * 
M T M G * 
G W G M * 
G G M T G 
W G G W M 

Opened nodes (order added):
[(1, 0), (0, 1), (1, 1), (0, 2), (1, 2), (0, 3), (1, 3), (0, 4), (1, 4), (2, 4), (3, 4), (2, 3), (4, 4), (3, 3), (5, 4), (4, 3), (6, 4), (5, 3)]

Closed nodes (order processed):
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]

Final path from S to G:
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]


In [35]:
cost

11.799999999999999